In [4]:
import pyarrow.parquet as pq
trips = pq.read_table('data/raw/rideshare/fhvhv_tripdata_2023-03.parquet')
trips = trips.to_pandas()
trips.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-03-01 00:02:05,2023-03-01 00:05:26,2023-03-01 00:05:26,2023-03-01 00:11:05,114,79,1.21,...,0.69,2.75,0.0,0.00,5.50,N,N,,N,N
1,HV0003,B03404,B03404,2023-03-01 00:11:47,2023-03-01 00:12:58,2023-03-01 00:14:30,2023-03-01 00:31:12,79,79,2.61,...,1.87,2.75,0.0,0.00,13.46,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-01 00:35:59,2023-03-01 00:36:50,2023-03-01 00:38:51,2023-03-01 01:05:46,113,265,6.58,...,0.00,0.00,0.0,0.00,25.19,N,N,,N,N
3,HV0003,B03404,B03404,2023-03-01 00:24:18,2023-03-01 00:30:55,2023-03-01 00:31:03,2023-03-01 00:49:24,229,260,4.28,...,1.52,0.75,0.0,4.00,15.54,Y,N,,N,N
4,HV0003,B03404,B03404,2023-03-01 00:04:54,2023-03-01 00:07:43,2023-03-01 00:08:00,2023-03-01 00:25:29,144,179,9.31,...,2.79,2.75,0.0,3.79,21.24,N,N,,N,N


In [3]:
len(trips)

20413539

In [20]:
trips['request_datetime'].dt.day.unique()

array([ 1, 28,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31])

### Trip data

In [1]:
import pyarrow.parquet as pq
from pyarrow import compute
import pandas as pd

In [70]:
# Time
month = '2023-03'
# Demand intetval
interval = 15

#### Yellow

In [5]:
filename = 'data/raw/rideshare/yellow_tripdata_' + month + '.parquet'

start = pd.Timestamp('2023-03-01')
end = pd.Timestamp('2023-03-07')

table = pq.read_table(filename, filters=[('tpep_pickup_datetime','>=',start),('tpep_pickup_datetime','<=',end)], use_threads=True)
df = table.to_pandas()

In [8]:
# Discrtize
df = df[['tpep_pickup_datetime','PULocationID','DOLocationID']]
df.loc[:,'interval'] = (df['tpep_pickup_datetime'].dt.hour*60 + df['tpep_pickup_datetime'].dt.minute)//interval
df_processed_yellow = df.groupby(['PULocationID','DOLocationID','interval']).count()

In [9]:
# Count
df_processed_yellow = df_processed_yellow.reset_index()
df_processed_yellow.columns = ['ori','dest','time','demand']
df_processed_yellow.head()

,ori,dest,time,demand
0,1,1,16,1
1,1,1,19,1
2,1,1,21,1
3,1,1,22,1
4,1,1,23,1


#### Green

In [11]:
filename = 'data/raw/rideshare/green_tripdata_' + month + '.parquet'

start = pd.Timestamp('2023-03-01')
end = pd.Timestamp('2023-03-07')

table = pq.read_table(filename, filters=[('lpep_pickup_datetime','>=',start),('lpep_pickup_datetime','<=',end)], use_threads=True)
df = table.to_pandas()

# Discrtize
df = df[['lpep_pickup_datetime','PULocationID','DOLocationID']]
df.loc[:,'interval'] = (df['lpep_pickup_datetime'].dt.hour*60 + df['lpep_pickup_datetime'].dt.minute)//interval
df_processed_green = df.groupby(['PULocationID','DOLocationID','interval']).count()

# Count
df_processed_green = df_processed_green.reset_index()
df_processed_green.columns = ['ori','dest','time','demand']
df_processed_green.head()

,ori,dest,time,demand
0,3,3,38,1
1,3,200,77,1
2,4,56,35,1
3,7,7,0,5
4,7,7,3,1


#### FHV

In [73]:
filename = 'data/raw/rideshare/fhv_tripdata_' + month + '.parquet'

start = pd.Timestamp('2023-03-01')
end = pd.Timestamp('2017-03-07')

table = pq.read_table(filename, filters=[('pickup_datetime','>=',start),('pickup_datetime','<=',end)], use_threads=True)
df = table.to_pandas()

In [74]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2017-01-01 00:30:00,1989-01-01,NaN,NaN,None,B00001
1,B00001,2017-01-01 00:30:00,1989-01-01,NaN,NaN,None,B00001
2,B00008,2017-01-01 00:50:00,1989-01-01,NaN,NaN,None,B00008
3,B00009,2017-01-01 00:10:00,1989-01-01,NaN,NaN,None,B00009
4,B00009,2017-01-01 00:50:00,1989-01-01,NaN,NaN,None,B00009


In [69]:
df[df['dispatching_base_num'].isin(['B03404', 'B03406', 'B02872', 'B02764', 'B02882', 'B02887',
       'B02869', 'B02870', 'B02835', 'B02871', 'B02765', 'B02877',
       'B02864', 'B02889'])]

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number


In [22]:
# Discrtize
df = df[['pickup_datetime','PUlocationID','DOlocationID']]
df.loc[:,'interval'] = (df['pickup_datetime'].dt.hour*60 + df['pickup_datetime'].dt.minute)//interval
df_processed_fhv = df.groupby(['PUlocationID','DOlocationID','interval']).count()

C:\Users\11481\AppData\Local\Temp\ipykernel_17392\1612595715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'interval'] = (df['pickup_datetime'].dt.hour*60 + df['pickup_datetime'].dt.minute)//interval


In [23]:
# Count
df_processed_fhv = df_processed_fhv.reset_index()
df_processed_fhv.columns = ['ori','dest','time','demand']
df_processed_fhv.head()

,ori,dest,time,demand
0,1.0,1.0,37,1
1,1.0,1.0,50,1
2,1.0,1.0,72,1
3,1.0,1.0,86,1
4,1.0,5.0,43,1


#### High volumn FHV

In [64]:
filename = 'data/raw/rideshare/fhvhv_tripdata_' + month + '.parquet'

start = pd.Timestamp('2023-03-01')
end = pd.Timestamp('2023-03-07')

table = pq.read_table(filename, filters=[('request_datetime','>=',start),('request_datetime','<=',end)], use_threads=True)
df = table.to_pandas()

In [16]:
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-03-01 00:02:05,2023-03-01 00:05:26,2023-03-01 00:05:26,2023-03-01 00:11:05,114,79,1.21,...,0.69,2.75,0.0,0.00,5.50,N,N,,N,N
1,HV0003,B03404,B03404,2023-03-01 00:11:47,2023-03-01 00:12:58,2023-03-01 00:14:30,2023-03-01 00:31:12,79,79,2.61,...,1.87,2.75,0.0,0.00,13.46,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-01 00:35:59,2023-03-01 00:36:50,2023-03-01 00:38:51,2023-03-01 01:05:46,113,265,6.58,...,0.00,0.00,0.0,0.00,25.19,N,N,,N,N
3,HV0003,B03404,B03404,2023-03-01 00:24:18,2023-03-01 00:30:55,2023-03-01 00:31:03,2023-03-01 00:49:24,229,260,4.28,...,1.52,0.75,0.0,4.00,15.54,Y,N,,N,N
4,HV0003,B03404,B03404,2023-03-01 00:04:54,2023-03-01 00:07:43,2023-03-01 00:08:00,2023-03-01 00:25:29,144,179,9.31,...,2.79,2.75,0.0,3.79,21.24,N,N,,N,N


In [17]:
df = df[['request_datetime','PULocationID','DOLocationID']]
df['interval'] = (df['request_datetime'].dt.hour*60 + df['request_datetime'].dt.minute)//interval
df_processed_fhvhv = df.groupby(['PULocationID','DOLocationID','interval']).count()

In [18]:
df_processed_fhvhv = df_processed_fhvhv.reset_index()
df_processed_fhvhv.columns = ['ori','dest','time','demand']
df_processed_fhvhv.head()

,ori,dest,time,demand
0,2,89,47,1
1,2,124,73,1
2,2,134,64,1
3,2,219,59,1
4,2,265,85,1


### Geo data

In [46]:
import geopandas as gpd
import numpy as np

zipfile = 'data/taxi_zones.shp'
zones = gpd.read_file(zipfile)
zones = zones.sort_values(['LocationID']).reset_index(drop=True)

In [47]:
zones.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."


In [48]:
# Zones in trip data but not in taxi zones
for each in df_processed.ori.unique():
    if each not in zones.LocationID.unique():
        print(each)

57
105
265


In [50]:
# Zones in taxi zones but not in trip data
for each in zones.LocationID.unique():
    if each not in df_processed.ori.unique():
        print(each)

1
103


In [43]:
zones_centroid = zones['geometry'].centroid
zones_polygon = zones['geometry']

#### Euclidean

In [55]:
euc_dist = zones_centroid.apply(lambda x:zones_centroid.distance(x)).to_numpy().flatten()

In [56]:
census_adj = pd.DataFrame(np.array([np.repeat(zones['LocationID'].to_numpy(), len(zones)),
                                    np.tile(zones['LocationID'].to_numpy(), len(zones)),
                                    euc_dist]).T, columns=['start_tract','end_tract','euc'])

In [62]:
# Threshold of the zones to be considered as connected 1
thres = 3000

census_adj.loc[(census_adj['euc']>0)&(census_adj['euc']<thres),'euc'] = 1
census_adj.loc[(census_adj['euc']>=thres),'euc'] = 3000/census_adj.loc[(census_adj['euc']>=thres),'euc']

#### Connectivity

In [69]:
touch = zones_polygon.apply(lambda x: zones_polygon.touches(x)).to_numpy(dtype=np.float32).flatten()

In [73]:
census_adj['con'] = touch

In [74]:
census_adj.head()

,start_tract,end_tract,euc,con
0,1.0,1.0,0.000000,0.0
1,1.0,2.0,0.030319,0.0
2,1.0,3.0,0.027231,0.0
3,1.0,4.0,0.053714,0.0
4,1.0,5.0,0.058982,0.0


### Explore processed data

In [2]:
df = pd.read_csv('data/processed/processed_data.csv')

In [4]:
df.columns

Index(['zone', 'bin', 'day', 'month', 'time_period', 'demand', 'air_temp',
       'air_temp_feel', 'rain', 'snow', 'residential', 'education',
       'recreational', 'commercial', 'health', 'subway_stations', 'bus_stops',
       'avg_demand_five', 'historical_average'],
      dtype='object')

In [9]:
df.head(10)

,zone,bin,day,month,time_period,demand,air_temp,air_temp_feel,rain,snow,residential,education,recreational,commercial,health,subway_stations,bus_stops,avg_demand_five,historical_average
0,4,0,3,6,off_peak,7.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,NaN,6.052632
1,4,1,3,6,off_peak,5.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,NaN,4.315789
2,4,2,3,6,off_peak,1.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,NaN,4.473684
3,4,3,3,6,off_peak,3.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,NaN,3.631579
4,4,4,3,6,off_peak,1.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,3.4,4.631579
5,4,5,3,6,off_peak,3.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,2.6,3.631579
6,4,6,3,6,off_peak,4.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,2.4,4.421053
7,4,7,3,6,off_peak,6.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,3.4,4.578947
8,4,8,3,6,off_peak,3.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,3.4,3.842105
9,4,9,3,6,off_peak,5.0,73.18,73.31,0.0,0.0,57,11,30,0,1,0.0,11.0,4.2,3.736842


In [12]:
df.groupby('day').apply(lambda x:len(x['zone'].unique()))

day
3     63
4     63
5     63
6     63
7     63
10    63
11    63
12    63
13    63
14    63
17    63
18    63
19    63
20    63
21    63
24    63
25    63
26    63
27    63
28    63
dtype: int64

In [14]:
df_demand = df[['zone','month','day','bin','demand']]
df_demand.sort_values(by=['month','day','bin','zone'])

,zone,month,day,bin,demand
0,4,6,3,0,7.0
5760,12,6,3,0,0.0
11520,13,6,3,0,1.0
17280,24,6,3,0,3.0
23040,41,6,3,0,10.0
...,...,...,...,...,...
339839,246,6,28,287,47.0
345599,249,6,28,287,42.0
351359,261,6,28,287,13.0
357119,262,6,28,287,2.0
